In [75]:
import pandas as pd
import numpy as np

df = pd.read_csv('dataset_formateado.csv')
df = df[df['Sale or Return'] == 'sale']
df

,Unnamed: 0,Date,Time,Item Code,Quantity Sold (kilo),Unit Selling Price (EUR/kg),Sale or Return,Discount (Yes/No),Item Name,Category Code,Category Name
0,0,2020-07-01,09:15:07,0,0.396,0.99,sale,No,Paopaojiao (Jingpin),1011010504,Capsicum
1,1,2020-07-01,09:17:27,1,0.849,0.42,sale,No,Chinese Cabbage,1011010101,Flower/Leaf Vegetables
2,2,2020-07-01,09:17:33,0,0.409,0.99,sale,No,Paopaojiao (Jingpin),1011010504,Capsicum
3,3,2020-07-01,09:19:45,2,0.421,1.30,sale,No,Shanghaiqing,1011010101,Flower/Leaf Vegetables
4,4,2020-07-01,09:20:23,3,0.539,1.04,sale,No,Caixin,1011010101,Flower/Leaf Vegetables
...,...,...,...,...,...,...,...,...,...,...,...
878498,878498,2023-06-30,21:35:13,67,0.284,3.12,sale,No,Xixia Black Mushroom (1),1011010801,Edible Mushroom
878499,878499,2023-06-30,21:35:14,93,0.669,1.56,sale,No,Changxianqie,1011010501,Solanum
878500,878500,2023-06-30,21:35:20,67,0.125,3.12,sale,No,Xixia Black Mushroom (1),1011010801,Edible Mushroom
878501,878501,2023-06-30,21:35:21,62,0.252,0.68,sale,No,Wuhu Green Pepper (1),1011010504,Capsicum


In [56]:
import pandas as pd
import numpy as np
from xgboost import XGBRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error

In [68]:
#Elegir producto del que quiere predecir el stock
producto = 'Paopaojiao (Jingpin)' 
#Elegir el tiempo de stock que quiere predecir
intervalo = 'Diariamente' #['Mensualmente', 'Anualmente','Diariamente']

In [69]:
#Seleccionar el producto del que se quiere predecir
df = df[df['Item Name'] == producto].reset_index()
df

,index,Unnamed: 0,Date,Time,Item Code,Quantity Sold (kilo),Unit Selling Price (EUR/kg),Sale or Return,Discount (Yes/No),Item Name,Category Code,Category Name
0,0,0,2020-07-01,09:15:07,0,0.396,0.99,sale,No,Paopaojiao (Jingpin),1011010504,Capsicum
1,2,2,2020-07-01,09:17:33,0,0.409,0.99,sale,No,Paopaojiao (Jingpin),1011010504,Capsicum
2,5,5,2020-07-01,09:21:55,0,0.277,0.99,sale,No,Paopaojiao (Jingpin),1011010504,Capsicum
3,7,7,2020-07-01,09:22:01,0,0.132,0.99,sale,No,Paopaojiao (Jingpin),1011010504,Capsicum
4,36,36,2020-07-01,09:41:09,0,0.198,0.99,sale,No,Paopaojiao (Jingpin),1011010504,Capsicum
...,...,...,...,...,...,...,...,...,...,...,...,...
25325,229136,229136,2021-02-10,18:31:06,0,0.234,1.56,sale,No,Paopaojiao (Jingpin),1011010504,Capsicum
25326,229164,229164,2021-02-10,18:35:31,0,0.417,1.56,sale,No,Paopaojiao (Jingpin),1011010504,Capsicum
25327,229925,229925,2021-02-13,10:00:09,0,0.259,1.56,sale,No,Paopaojiao (Jingpin),1011010504,Capsicum
25328,232336,232336,2021-02-15,10:40:23,0,0.417,1.56,sale,No,Paopaojiao (Jingpin),1011010504,Capsicum


In [70]:
# 📌 2. Procesar fecha y Fragmentar fecha y hora 
df['Date'] = pd.to_datetime(df['Date'])
df['Time'] = pd.to_datetime(df['Time'])

df["year"] = df["Date"].dt.year
df["month"] = df["Date"].dt.month
df["day"] = df["Date"].dt.day
# df["weekday"] = df["Date"].dt.weekday  # 0 = Lunes, 6 = Domingo
df["hour"] = df["Time"].dt.hour
df["week"] = df["Date"].dt.isocalendar().week
df.head()

C:\Users\santi\AppData\Local\Temp\ipykernel_6372\3862880160.py:3: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['Time'] = pd.to_datetime(df['Time'])


,index,Unnamed: 0,Date,Time,Item Code,Quantity Sold (kilo),Unit Selling Price (EUR/kg),Sale or Return,Discount (Yes/No),Item Name,Category Code,Category Name,year,month,day,hour,week
0,0,0,2020-07-01,2025-02-13 09:15:07,0,0.396,0.99,sale,No,Paopaojiao (Jingpin),1011010504,Capsicum,2020,7,1,9,27
1,2,2,2020-07-01,2025-02-13 09:17:33,0,0.409,0.99,sale,No,Paopaojiao (Jingpin),1011010504,Capsicum,2020,7,1,9,27
2,5,5,2020-07-01,2025-02-13 09:21:55,0,0.277,0.99,sale,No,Paopaojiao (Jingpin),1011010504,Capsicum,2020,7,1,9,27
3,7,7,2020-07-01,2025-02-13 09:22:01,0,0.132,0.99,sale,No,Paopaojiao (Jingpin),1011010504,Capsicum,2020,7,1,9,27
4,36,36,2020-07-01,2025-02-13 09:41:09,0,0.198,0.99,sale,No,Paopaojiao (Jingpin),1011010504,Capsicum,2020,7,1,9,27


In [71]:
# 📌 3. Agrupar por producto y fecha para obtener total vendido por el intervalo seleccionado
if intervalo == 'Anualmente':
    intervalo_ft = ['year']
elif intervalo == 'Mensualmente':
    intervalo_ft = ['year', 'month']
else:
    intervalo_ft = ['year', 'month', 'day']

    
df_grouped = df.groupby(intervalo_ft+["Item Name"]).agg(
    {"Quantity Sold (kilo)": "sum", "Unit Selling Price (EUR/kg)": "mean"}
).reset_index()
df_grouped

,year,month,day,Item Name,Quantity Sold (kilo),Unit Selling Price (EUR/kg)
0,2020,7,1,Paopaojiao (Jingpin),54.548,0.99
1,2020,7,2,Paopaojiao (Jingpin),39.810,0.99
2,2020,7,3,Paopaojiao (Jingpin),32.351,0.99
3,2020,7,4,Paopaojiao (Jingpin),45.815,0.99
4,2020,7,5,Paopaojiao (Jingpin),53.874,0.99
...,...,...,...,...,...,...
223,2021,2,9,Paopaojiao (Jingpin),12.520,1.56
224,2021,2,10,Paopaojiao (Jingpin),1.327,1.56
225,2021,2,13,Paopaojiao (Jingpin),0.259,1.56
226,2021,2,15,Paopaojiao (Jingpin),0.417,1.56


In [72]:
# 📌 4. Selección de variables
features = intervalo_ft + [ "Unit Selling Price (EUR/kg)"]

X = df_grouped[features]
y = df_grouped["Quantity Sold (kilo)"]

# # 📌 5. División en entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# # 📌 6. Entrenamiento del modelo
modelo = XGBRegressor(n_estimators=23, learning_rate=0.1)
modelo.fit(X_train, y_train)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=0.1, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=None, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=23, n_jobs=None,
             num_parallel_tree=None, random_state=None, ...)

In [84]:
# 📌 7. Evaluación
preds = modelo.predict(X_test)
mae = mean_absolute_error(y_test, preds)
print(f"Error MAE: {mae}")

# 📌 8. Predicción para una fecha futura

# fecha_prediccion = 
nueva_fecha = pd.DataFrame([[2023,7,1, 1.5]], columns=X.columns)
prediccion = modelo.predict(nueva_fecha)
print(f"Predicción de ventas: {prediccion[0]} kilos")

#Las predicciones no tienen mucho sentido, cuanto más caro el producto, más se vende.

Error MAE: 11.263131206678308
Predicción de ventas: 59.3410530090332 kilos
